In [1]:
%matplotlib inline
import librosa
import librosa.display
import IPython
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import re
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram

In [3]:
import os

folder_path = '/content/drive/MyDrive/музыка'  # Путь к папке с музыкой
files = os.listdir(folder_path)

df = pd.DataFrame(columns=['song','mean_chroma_stft','mean_spec_cent','mean_spec_bw','mean_rolloff','mean_zcr','mean_mfcc'])

for file in files:
    file_path = os.path.join(folder_path, file)
    y, sr = librosa.load(file_path)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    features = [np.mean(f) for f in [chroma_stft, spec_cent, spec_bw, rolloff, zcr, mfcc]]
    df.loc[len(df)] = [file, *features]


In [4]:
aggregated_data = {
    'average': df.mean(),
    'std_dev': df.std()
}

aggregated_df = pd.DataFrame(aggregated_data)
aggregated_df

<ipython-input-4-8dc2da0d9820>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  'average': df.mean(),
<ipython-input-4-8dc2da0d9820>:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  'std_dev': df.std()


,average,std_dev
mean_chroma_stft,0.364383,0.051213
mean_spec_cent,2329.067634,375.574498
mean_spec_bw,2455.963837,236.411737
mean_rolloff,4930.637650,832.446332
mean_zcr,0.101505,0.024134
mean_mfcc,1.975106,3.355399


In [5]:
X=df[['mean_chroma_stft','mean_spec_bw']]
silhouette_scores=[]
for k in range(2,11):
  model = KMeans(n_clusters = k,n_init='auto')
  model.fit(X)
  clusters = model.predict(X)
  print(k,'--',silhouette_score(X,clusters))

2 -- 0.6153950093404011
3 -- 0.6651671152434958
4 -- 0.6069620023797325
5 -- 0.5854903068326527
6 -- 0.48194669235608717
7 -- 0.5036892970713911
8 -- 0.5277894827438688
9 -- 0.5888491266670951
10 -- 0.5682583543553045


In [6]:
import random
song_clusters = pd.DataFrame({'song': df['song'], 'cluster': clusters})
fig = px.scatter(x=X['mean_chroma_stft'], y=X['mean_spec_bw'], hover_data=[song_clusters['song']])
fig.update_traces(marker=dict(size=25))
fig.show()

In [7]:
model = KMeans(n_clusters=3)
model.fit(X)
clusters = model.predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [18]:
fig = px.scatter(x=X['mean_chroma_stft'], y=X['mean_spec_bw'], color=clusters, hover_data=[song_clusters['song']])
fig.update_traces(marker=dict(size=25))
fig.show()